In [1]:
from sklearn.datasets import load_diabetes
import pickle
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
import os
import torch


from synthcity.metrics.eval_performance import (
    PerformanceEvaluatorMLP,
    PerformanceEvaluatorXGB,
)
from synthcity.utils import reproducibility
from synthcity.plugins import Plugins
import synthcity.logger as log
from synthcity.plugins.core.dataloader import GenericDataLoader
from DGE_utils import metric_different_datasets, mean_across_pandas, add_std, get_folder_names

reproducibility.clear_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


assert device.type == 'cuda'
Plugins(categories=["generic"]).list()

[2023-12-04T10:51:04.223735+0000][8295][CRITICAL] load failed: 
GReaT is not installed. Please install it with pip install GReaT.
Please be aware that GReaT is only available for python >= 3.9.

[2023-12-04T10:51:04.224919+0000][8295][CRITICAL] load failed: module 'synthcity.plugins.generic.plugin_great' has no attribute 'plugin'
[2023-12-04T10:51:04.225768+0000][8295][CRITICAL] module plugin_great load failed
[2023-12-04T10:51:04.233719+0000][8295][CRITICAL] module disabled: /home/JacquelineYau/deep_generative_ensemble/synthcity/src/synthcity/plugins/generic/plugin_goggle.py


['ctgan',
 'rtvae',
 'marginal_distributions',
 'arf',
 'uniform_sampler',
 'dummy_sampler',
 'bayesian_network',
 'nflow',
 'tvae',
 'ddpm']

In [2]:
# Set up params for boosting experiment
from DGE_data import get_real_and_synthetic, get_real_and_synthetic_with_multiple_models

# let's restrict ourselves to classification datasets
datasets = ['moons', 'circles', 'breast_cancer',
            'adult', 'covid']
# ['moons', 'circles','cal_housing', 'adult', 'diabetes', 'breast_cancer',  'seer', 'cutract' ]
model_name = 'ctgan'  # synthetic data model
# model_names = ['tvae', 'nflow', 'ctgan', 'adsgan']
# model_name = model_names[0]
# for m in model_names[1:]:
#     model_name += "_" + m

n_models = 20  # number of models in ensemble, for each run. 20
max_n = 2000  # maximum number of data points to use for training generative model.
nsyn = 2000  # number of synthetic data points per synthetic dataset. Defaults to same as generative training size if None

num_runs = 3 # Number of runs. Don't choose to large, since total number of synthetic datasets is num_runs*n_models. 10

# Per section 4.1, 10 runs with different seeds

# Whether to load and save models and synthetic datasets
load = True  # results
load_syn = True  # data
save = True  # save results and data

verbose = True

In [3]:
# Run boosting experiment
from DGE_experiments import predictive_experiment, predictive_experiment_stacking, boosting_DGE
import pandas as pd
import time

all_means = {}
all_stds = {}

#datasets = ['moons', 'circles', 'breast_cancer', 'adult', 'covid']
datasets = ['covid', 'breast_cacner']
test_only_dataset = ['adult']
#model_names = ['ctgan']
#num_runs=1
#model_type = ['deepish_mlp', 'largest_mlp', 'rf', 'svm', 'xgboost', 'knn']
model_type = 'deepish_mlp'
#mixed_models = True
#meta_model='lr'
boosting = "SAMME"

print("Model to run: ", model_name)
print("Downstream classifier model type: ", model_type)
print("boosting method: ", boosting)
print("n_models: ", n_models)
print("num_runs: ", num_runs)
print("datasets: ", datasets)
print("model string: ", model_name)
print("verbose: ", verbose)

start_time = time.time()
for dataset in datasets:
    workspace_folder, results_folder = get_folder_names(
        dataset, model_name, max_n=max_n, nsyn=nsyn)
    # For toy runs
    workspace_folder = f"{workspace_folder}_{boosting}_retry"
    results_folder = f"{results_folder}_{boosting}_retry"

    print(f'Dataset {dataset}\n')
    print("workspace_folder: ", workspace_folder)
    print("results_folder: ", results_folder)

    means, stds, _ = boosting_DGE(dataset, model_name, num_runs=num_runs, num_iter=n_models, boosting=boosting, p_train=0.8, 
                                  max_n=max_n, nsyn=nsyn, reduce_to=20000, task_type=model_type, workspace_folder=workspace_folder, 
                                  save=save, load=load, verbose=verbose)
    print("printing weighted avg means to latex:")
    print(means.to_latex())
    print("printing weighted stds to latex:")
    print(stds.to_latex())

    all_means[dataset] = means
    all_stds[dataset] = stds

    print("size of means: ", means.shape)
    print("mean elements: ", means)

end_time = time.time()
time_elapsed = end_time - start_time
print("Time it took to run the experiment: ", time_elapsed)

Model to run:  ctgan
Downstream classifier model type:  deepish_mlp
boosting method:  SAMME
n_models:  20
num_runs:  3
datasets:  ['covid', 'breast_cacner']
model string:  ctgan
verbose:  True
Dataset covid

workspace_folder:  workspace/covid/ctgan/nmax_2000_nsyn_2000_SAMME_retry
results_folder:  results/covid_ctgan_nmax_2000_nsyn_2000_SAMME_retry
Boosting DGE


[2023-12-04T10:51:19.791297+0000][8295][CRITICAL] load failed: module 'synthcity.plugins.generic.plugin_great' has no attribute 'plugin'
[2023-12-04T10:51:19.792388+0000][8295][CRITICAL] load failed: module 'synthcity.plugins.generic.plugin_great' has no attribute 'plugin'
[2023-12-04T10:51:19.793083+0000][8295][CRITICAL] module plugin_great load failed
[2023-12-04T10:51:19.794157+0000][8295][CRITICAL] module disabled: /home/JacquelineYau/deep_generative_ensemble/synthcity/src/synthcity/plugins/generic/plugin_goggle.py
[2023-12-04T10:51:19.939132+0000][8295][CRITICAL] load failed: libcudart.so.12: cannot open shared object file: No such file or directory
[2023-12-04T10:51:19.940135+0000][8295][CRITICAL] load failed: module 'synthcity.plugins.privacy.plugin_decaf' has no attribute 'plugin'
[2023-12-04T10:51:19.940921+0000][8295][CRITICAL] module plugin_decaf load failed


np.unique(y):  [1 2]
n_train: 2000, nsyn: 2000
targettype:  classification
Run 1 / 3
Boosting iter: 1 / 20
Boost generative model training and synthetic data generation
Training model 1/20
random state: 0
self._plugins:  {'uniform_sampler': <class 'synthcity.plugins.generic.plugin_uniform_sampler.UniformSamplerPlugin'>, 'bayesian_network': <class 'synthcity.plugins.generic.plugin_bayesian_network.BayesianNetworkPlugin'>, 'ctgan': <class 'synthcity.plugins.generic.plugin_ctgan.CTGANPlugin'>, 'marginal_distributions': <class 'synthcity.plugins.generic.plugin_marginal_distributions.MarginalDistributionPlugin'>, 'dummy_sampler': <class 'synthcity.plugins.generic.plugin_dummy_sampler.DummySamplerPlugin'>, 'arf': <class 'synthcity.plugins.generic.plugin_arf.ARFPlugin'>, 'ddpm': <class 'synthcity.plugins.generic.plugin_ddpm.TabDDPMPlugin'>, 'rtvae': <class 'synthcity.plugins.generic.plugin_rtvae.RTVAEPlugin'>, 'nflow': <class 'synthcity.plugins.generic.plugin_nflow.NormalizingFlowsPlugin'>, '

 52%|██████████████████████████████████████████████████████████████████████████████████████▌                                                                              | 1049/2000 [07:20<06:39,  2.38it/s]
[2023-12-04T10:58:44.594359+0000][8295][CRITICAL] load failed: module 'synthcity.plugins.generic.plugin_great' has no attribute 'plugin'
[2023-12-04T10:58:44.595970+0000][8295][CRITICAL] load failed: module 'synthcity.plugins.generic.plugin_great' has no attribute 'plugin'
[2023-12-04T10:58:44.597101+0000][8295][CRITICAL] module plugin_great load failed
[2023-12-04T10:58:44.598576+0000][8295][CRITICAL] module disabled: /home/JacquelineYau/deep_generative_ensemble/synthcity/src/synthcity/plugins/generic/plugin_goggle.py
[2023-12-04T10:58:44.602106+0000][8295][CRITICAL] load failed: module 'synthcity.plugins.privacy.plugin_decaf' has no attribute 'plugin'
[2023-12-04T10:58:44.605202+0000][8295][CRITICAL] load failed: module 'synthcity.plugins.privacy.plugin_decaf' has no attribute 'pl

fileroot in aggregate:  workspace/covid/ctgan/nmax_2000_nsyn_2000_SAMME_retry/supervised_task_deepish_mlp
task function name:  supervised_task
task type:  deepish_mlp
Saving model as workspace/covid/ctgan/nmax_2000_nsyn_2000_SAMME_retry/supervised_task_deepish_mlp_DGE_0__0.pkl
loading existing model...
supervised task:  classification
model type:  deepish_mlp
shape of pred in supervised task:  (2000,)
Boosting iter 0 accuracy:  0.072
boost with SAMME
normalized data_weights:  [0.00049991 0.00049991 0.00049991 ... 0.00049991 0.00049991 0.00049991]
are all weights equal?:  0
estimator weight:  -0.0025563656137701446
Boosting iter: 2 / 20
Boost generative model training and synthetic data generation
Training model 2/20
random state: 1
self._plugins:  {'uniform_sampler': <class 'synthcity.plugins.generic.plugin_uniform_sampler.UniformSamplerPlugin'>, 'bayesian_network': <class 'synthcity.plugins.generic.plugin_bayesian_network.BayesianNetworkPlugin'>, 'ctgan': <class 'synthcity.plugins.gen

...

 57%|██████████████████████████████████████████████████████████████████████████████████████████████▊                                                                      | 1149/2000 [09:27<06:59,  2.03it/s]
[2023-12-04T17:34:29.075828+0000][8295][CRITICAL] load failed: module 'synthcity.plugins.generic.plugin_great' has no attribute 'plugin'
[2023-12-04T17:34:29.077893+0000][8295][CRITICAL] load failed: module 'synthcity.plugins.generic.plugin_great' has no attribute 'plugin'
[2023-12-04T17:34:29.079652+0000][8295][CRITICAL] module plugin_great load failed
[2023-12-04T17:34:29.081734+0000][8295][CRITICAL] module disabled: /home/JacquelineYau/deep_generative_ensemble/synthcity/src/synthcity/plugins/generic/plugin_goggle.py
[2023-12-04T17:34:29.085505+0000][8295][CRITICAL] load failed: module 'synthcity.plugins.privacy.plugin_decaf' has no attribute 'plugin'
[2023-12-04T17:34:29.087589+0000][8295][CRITICAL] load failed: module 'synthcity.plugins.privacy.plugin_decaf' has no attribute 'pl

fileroot in aggregate:  workspace/covid/ctgan/nmax_2000_nsyn_2000_SAMME_retry/supervised_task_deepish_mlp
task function name:  supervised_task
task type:  deepish_mlp
Saving model as workspace/covid/ctgan/nmax_2000_nsyn_2000_SAMME_retry/supervised_task_deepish_mlp_DGE_18__0.pkl
loading existing model...
supervised task:  classification
model type:  deepish_mlp
shape of pred in supervised task:  (2000,)
Boosting iter 18 accuracy:  0.91
boost with SAMME
normalized data_weights:  [0.0004981  0.0004981  0.00049925 ... 0.00051013 0.00050641 0.0004981 ]
are all weights equal?:  0
estimator weight:  0.0022960186511272436
Boosting iter: 20 / 20
Boost generative model training and synthetic data generation
Training model 60/20
random state: 59
self._plugins:  {'uniform_sampler': <class 'synthcity.plugins.generic.plugin_uniform_sampler.UniformSamplerPlugin'>, 'bayesian_network': <class 'synthcity.plugins.generic.plugin_bayesian_network.BayesianNetworkPlugin'>, 'ctgan': <class 'synthcity.plugins.

 37%|██████████████████████████████████████████████████████████████▏                                                                                                       | 749/2000 [06:04<10:09,  2.05it/s]


fileroot in aggregate:  workspace/covid/ctgan/nmax_2000_nsyn_2000_SAMME_retry/supervised_task_deepish_mlp
task function name:  supervised_task
task type:  deepish_mlp
Saving model as workspace/covid/ctgan/nmax_2000_nsyn_2000_SAMME_retry/supervised_task_deepish_mlp_DGE_19__0.pkl
loading existing model...
supervised task:  classification
model type:  deepish_mlp
shape of pred in supervised task:  (2000,)
Boosting iter 19 accuracy:  0.9085
boost with SAMME
normalized data_weights:  [0.000498   0.000498   0.00049914 ... 0.00051119 0.00050746 0.000498  ]
are all weights equal?:  0
estimator weight:  0.002275374070917601
Finished run 2 / 3
Start final evaluation...
run:  0
fileroot in aggregate:  workspace/covid/ctgan/nmax_2000_nsyn_2000_SAMME_retry/supervised_task_deepish_mlp
task function name:  supervised_task
task type:  deepish_mlp
Saving model as workspace/covid/ctgan/nmax_2000_nsyn_2000_SAMME_retry/supervised_task_deepish_mlp_oracle_run_0__0.pkl
Train model in aggregate
Train model 1/

ValueError: y_prob contains values greater than 1.

In [ ]:
# Metrics
# Print results, aggregated over different datasets
means_consolidated = metric_different_datasets(all_means, to_print=False)
if num_runs>1:
    stds_consolidated = metric_different_datasets(all_stds, to_print=False)
    stds_consolidated.drop(columns=['Mean'], inplace=True)
    print(add_std(means_consolidated, stds_consolidated).to_latex())
else:
    print(means_consolidated.to_latex())